In [1]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
import matplotlib.gridspec as gridspec
cudnn.benchmark = True
from sklearn.metrics import confusion_matrix

In [2]:
data_transforms = {
    'test': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'E:\datasets\dataset_noweather'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['test']}
class_names = image_datasets['test'].classes

In [3]:
device = torch.device("cpu")

In [5]:
model_ft = models.resnet152()

num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 11)
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-resnet152-bestloss-no_weather.pth',map_location ='cpu'))

<All keys matched successfully>

In [7]:
nb_classes = 11

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        from sklearn.metrics import classification_report

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])


ValueError: Number of classes, 11, does not match size of target_names, 10. Try specifying the labels parameter

In [8]:
target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния', 'Нет погоды' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.76      0.73      0.75       101
     облачно       0.72      0.66      0.69       130
       туман       0.87      0.88      0.87       148
       мороз       0.82      0.80      0.81       165
        град       0.94      0.74      0.83        65
      молния       0.88      0.98      0.92        44
  Нет погоды       0.85      0.80      0.83        82
       дождь       0.71      0.78      0.74       177
      радуга       0.96      0.90      0.93        30
        снег       0.70      0.80      0.75       163
      восход       0.95      0.85      0.90        71

    accuracy                           0.79      1176
   macro avg       0.83      0.81      0.82      1176
weighted avg       0.80      0.79      0.79      1176

